In [1]:
!pip install shapely

In [2]:
!pip install ggplot

In [3]:
from pyspark.sql import Row 
from pyspark.sql.types import *
import pandas as pdd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings
import os
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [4]:
Train = spark.read.csv("/FileStore/tables/train.csv",header=True, inferSchema=True)
Train.show(2)

+-------------------+--------------+--------------------+---------+----------+--------------+------------------+-----------------+----------------+
 Dates| Category| Descript|DayOfWeek|PdDistrict| Resolution| Address| X| Y|
+-------------------+--------------+--------------------+---------+----------+--------------+------------------+-----------------+----------------+
2015-05-13 23:53:00| WARRANTS| WARRANT ARREST|Wednesday| NORTHERN|ARREST, BOOKED|OAK ST / LAGUNA ST|-122.425891675136|37.7745985956747|
2015-05-13 23:53:00|OTHER OFFENSES|TRAFFIC VIOLATION...|Wednesday| NORTHERN|ARREST, BOOKED|OAK ST / LAGUNA ST|-122.425891675136|37.7745985956747|
+-------------------+--------------+--------------------+---------+----------+--------------+------------------+-----------------+----------------+
only showing top 2 rows

In [5]:
Train.printSchema()

root
-- Dates: timestamp (nullable = true)
-- Category: string (nullable = true)
-- Descript: string (nullable = true)
-- DayOfWeek: string (nullable = true)
-- PdDistrict: string (nullable = true)
-- Resolution: string (nullable = true)
-- Address: string (nullable = true)
-- X: double (nullable = true)
-- Y: double (nullable = true)

In [6]:
df = Train.select( "Category", "PdDistrict", "Address",Train.X.cast("double"), Train.Y.cast("double"), "Dates","DayOfWeek")
df.printSchema()

root
-- Category: string (nullable = true)
-- PdDistrict: string (nullable = true)
-- Address: string (nullable = true)
-- X: double (nullable = true)
-- Y: double (nullable = true)
-- Dates: timestamp (nullable = true)
-- DayOfWeek: string (nullable = true)

In [7]:
Q1_x, Q3_x = df.approxQuantile("X", [0.25, 0.75], 0.0)
x_min_range = Q1_x - (Q3_x - Q1_x)*1.5
x_max_range = Q3_x + (Q3_x - Q1_x)*1.5

Q1_y, Q3_y = df.approxQuantile("Y", [0.25, 0.75], 0.0)
y_min_range = Q1_y - (Q3_y - Q1_y)*1.5
y_max_range = Q3_y + (Q3_y - Q1_y)*1.5

df = df[df["X"] <= x_max_range]
df = df[df["X"] >= x_min_range]
df = df[df["Y"] <= y_max_range]
df = df[df["Y"] >= y_min_range]

In [8]:
df.groupBy("Category").count().orderBy("Category").show()

+--------------------+------+
 Category| count|
+--------------------+------+
 ARSON| 1385|
 ASSAULT| 72098|
 BAD CHECKS| 377|
 BRIBERY| 273|
 BURGLARY| 33429|
 DISORDERLY CONDUCT| 4166|
DRIVING UNDER THE...| 1990|
 DRUG/NARCOTIC| 52851|
 DRUNKENNESS| 4041|
 EMBEZZLEMENT| 1082|
 EXTORTION| 219|
 FAMILY OFFENSES| 456|
FORGERY/COUNTERFE...| 9595|
 FRAUD| 15138|
 GAMBLING| 137|
 KIDNAPPING| 2170|
 LARCENY/THEFT|163120|
 LIQUOR LAWS| 1784|
 LOITERING| 1204|
 MISSING PERSON| 24104|
+--------------------+------+
only showing top 20 rows

In [9]:
df.groupBy("PdDistrict").count().orderBy("PdDistrict").show()


+----------+------+
PdDistrict| count|
+----------+------+
 BAYVIEW| 89355|
 CENTRAL| 85455|
 INGLESIDE| 78841|
 MISSION|119898|
 NORTHERN|105169|
 PARK| 49269|
 RICHMOND| 26621|
 SOUTHERN|155542|
 TARAVAL| 25833|
TENDERLOIN| 81794|
+----------+------+

In [10]:
df_X_all = df.groupBy("Category").count().orderBy("count").toPandas()
fig = plt.figure(figsize=(25,12))
sb.set(font_scale=0.8, style = "whitegrid")
sb.barplot(x= "count", y = "Category", data = df_X_all)
plt.legend(loc='upper right')
display(fig)

In [11]:

df.createTempView("sf_crime")

In [12]:
#Spark SQL based
crime_cat = spark.sql("SELECT Category, COUNT(*) AS Numbers_Crime \
                       FROM sf_crime \
                       GROUP BY Category \
                       ORDER BY Numbers_Crime DESC")

crime_cat.show()

+--------------------+-------------+
 Category|Numbers_Crime|
+--------------------+-------------+
 LARCENY/THEFT| 163120|
 OTHER OFFENSES| 118452|
 NON-CRIMINAL| 85169|
 ASSAULT| 72098|
 DRUG/NARCOTIC| 52851|
 VEHICLE THEFT| 47861|
 WARRANTS| 40997|
 VANDALISM| 40127|
 BURGLARY| 33429|
 SUSPICIOUS OCC| 28383|
 MISSING PERSON| 24104|
 ROBBERY| 21914|
 FRAUD| 15138|
FORGERY/COUNTERFE...| 9595|
 SECONDARY CODES| 9088|
 WEAPON LAWS| 8094|
 PROSTITUTION| 7393|
 TRESPASS| 6971|
 STOLEN PROPERTY| 4328|
 DISORDERLY CONDUCT| 4166|
+--------------------+-------------+
only showing top 20 rows

In [13]:
from pyspark.sql.functions import count
crime_cat_df=df.groupBy("Category").agg(count("*").alias("Numbers_Crime")).orderBy("Numbers_Crime",ascending=False)
crime_cat_df.show()


+--------------------+-------------+
 Category|Numbers_Crime|
+--------------------+-------------+
 LARCENY/THEFT| 163120|
 OTHER OFFENSES| 118452|
 NON-CRIMINAL| 85169|
 ASSAULT| 72098|
 DRUG/NARCOTIC| 52851|
 VEHICLE THEFT| 47861|
 WARRANTS| 40997|
 VANDALISM| 40127|
 BURGLARY| 33429|
 SUSPICIOUS OCC| 28383|
 MISSING PERSON| 24104|
 ROBBERY| 21914|
 FRAUD| 15138|
FORGERY/COUNTERFE...| 9595|
 SECONDARY CODES| 9088|
 WEAPON LAWS| 8094|
 PROSTITUTION| 7393|
 TRESPASS| 6971|
 STOLEN PROPERTY| 4328|
 DISORDERLY CONDUCT| 4166|
+--------------------+-------------+
only showing top 20 rows

In [14]:
crime_category_pd =  crime_cat_df.toPandas()


/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [15]:
sb.set(style = "whitegrid")
fig = plt.figure(figsize=(25,20))
x = np.arange(len(crime_category_pd["Category"]))
plt.bar(x, crime_category_pd["Numbers_Crime"], color = sb.color_palette("PiYG", len(x)), data = crime_category_pd)
plt.ylabel("number of crimes", fontsize=20)
plt.xticks(x, tuple(crime_category_pd["Category"]), fontsize = 10, rotation = 90)
plt.title('number of crimes for different category', fontsize=20)
display(fig)

In [16]:

df_district = spark.sql("SELECT PdDistrict, COUNT(*) as Numbers_Crime \
                         FROM sf_crime \
                         GROUP BY PdDistrict \
                         ORDER BY Numbers_Crime DESC")
df_district.show()

+----------+-------------+
PdDistrict|Numbers_Crime|
+----------+-------------+
 SOUTHERN| 155542|
 MISSION| 119898|
 NORTHERN| 105169|
 BAYVIEW| 89355|
 CENTRAL| 85455|
TENDERLOIN| 81794|
 INGLESIDE| 78841|
 PARK| 49269|
 RICHMOND| 26621|
 TARAVAL| 25833|
+----------+-------------+

In [17]:
crime_PdDis_df=df.groupBy("PdDistrict").agg(count("*").alias("Numbers_Crime")).orderBy("Numbers_Crime",ascending=False)
crime_PdDis_df.show()

+----------+-------------+
PdDistrict|Numbers_Crime|
+----------+-------------+
 SOUTHERN| 155542|
 MISSION| 119898|
 NORTHERN| 105169|
 BAYVIEW| 89355|
 CENTRAL| 85455|
TENDERLOIN| 81794|
 INGLESIDE| 78841|
 PARK| 49269|
 RICHMOND| 26621|
 TARAVAL| 25833|
+----------+-------------+

In [18]:
crime_PdDis_pd =  crime_PdDis_df.toPandas()


/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [19]:
sb.set(style = "whitegrid")
fig = plt.figure(figsize=(25,20))
x = np.arange(len(crime_PdDis_pd["PdDistrict"]))
plt.bar(x, crime_PdDis_pd["Numbers_Crime"], color = sb.color_palette("PiYG", len(x)), data = crime_PdDis_pd)
plt.ylabel("number of crimes", fontsize=20)
plt.xticks(x, tuple(crime_PdDis_pd["PdDistrict"]), fontsize = 10, rotation = 90)
plt.title('number of crimes for different PdDistrict', fontsize=20)
display(fig)

In [20]:
from pyspark.sql.types import FloatType, BooleanType
from pyspark.sql.functions import udf
from shapely.geometry import Polygon, Point

In [21]:
poly = Polygon([(37.797844, -122.407050), (37.798937, -122.398205), (37.798519, -122.397808), 
              (37.795541, -122.396759), (37.794233, -122.395145), (37.786599, -122.404677),
              (37.797844, -122.407050)])  # this is when I know the order of the coordinates

x,y = poly.exterior.xy
fig = plt.figure(figsize=(4,2))
ax = fig.add_subplot(111)
ax.plot(x, y, color='blue', alpha=0.7, linewidth=1)
ax.set_title('Polygon')
display(fig)

In [22]:
def in_sf_downtown(x,y):
  point = Point(float(y), float(x))
  return poly.contains(point)


sf_downtown_udf = udf(lambda x, y: in_sf_downtown(x,y), StringType())


In [23]:
from pyspark.sql.functions import to_timestamp, to_date

df_Q3 = df.select(["X", "Y", sf_downtown_udf("X","Y").alias('In_downtown'), to_date("Dates").alias('Date'),"DayOfWeek"])
df_Q3.createOrReplaceTempView("sf_crime_Q3")
df_Q3.show(5)

+-------------------+------------------+-----------+----------+---------+
 X| Y|In_downtown| Date|DayOfWeek|
+-------------------+------------------+-----------+----------+---------+
 -122.425891675136| 37.7745985956747| false|2015-05-13|Wednesday|
 -122.425891675136| 37.7745985956747| false|2015-05-13|Wednesday|
 -122.42436302145| 37.8004143219856| false|2015-05-13|Wednesday|
-122.42699532676599| 37.80087263276921| false|2015-05-13|Wednesday|
 -122.438737622757|37.771541172057795| false|2015-05-13|Wednesday|
+-------------------+------------------+-----------+----------+---------+
only showing top 5 rows

In [24]:
from pyspark.sql.functions import  col, column, expr

crime_sunday_df=df_Q3.where(col("DayOfWeek") == "Sunday").where(col("In_downtown") == 'true').select('*')
crime_sunday_df.show()

+-------------------+------------------+-----------+----------+---------+
 X| Y|In_downtown| Date|DayOfWeek|
+-------------------+------------------+-----------+----------+---------+
 -122.405800540862| 37.7921233434727| true|2015-05-03| Sunday|
 -122.405800540862| 37.7921233434727| true|2015-05-03| Sunday|
-122.40610663336099| 37.79361439203321| true|2015-05-03| Sunday|
-122.40610663336099| 37.79361439203321| true|2015-05-03| Sunday|
 -122.402043557609| 37.7971874633378| true|2015-05-03| Sunday|
-122.39858448694099|37.797986280621295| true|2015-05-03| Sunday|
 -122.404013943546| 37.7897518225049| true|2015-05-03| Sunday|
-122.40552479680001| 37.7977714997338| true|2015-05-03| Sunday|
-122.40552479680001| 37.7977714997338| true|2015-05-03| Sunday|
-122.40552479680001| 37.7977714997338| true|2015-05-03| Sunday|
-122.40583874581002| 37.7973631784416| true|2015-04-19| Sunday|
 -122.403535460378| 37.7878244847404| true|2015-04-19| Sunday|
-122.40031605728599| 37.790099627367| true|2015-04-19| Sunday|
 -122.402163713618| 37.7895710255863| true|2015-04-19| Sunday|
 -122.400922761049| 37.794959803128| true|2015-04-19| Sunday|
-122.40377224998899| 37.78912092500261| true|2015-04-19| Sunday|
 -122.400344691288| 37.7902143401629| true|2015-04-19| Sunday|
-122.40398013244801| 37.7964121687875| true|2015-04-19| Sunday|
 -122.396556426173| 37.7941169057941| true|2015-04-05| Sunday|
 -122.396556426173| 37.7941169057941| true|2015-04-05| Sunday|
+-------------------+------------------+-----------+----------+---------+
only showing top 20 rows

In [25]:
df_crime_sunday = spark.sql("SELECT *\
                             FROM sf_crime_Q3 \
                             WHERE DayOfWeek = 'Sunday' and In_downtown = true")
df_crime_sunday.show()

+-------------------+------------------+-----------+----------+---------+
 X| Y|In_downtown| Date|DayOfWeek|
+-------------------+------------------+-----------+----------+---------+
 -122.405800540862| 37.7921233434727| true|2015-05-03| Sunday|
 -122.405800540862| 37.7921233434727| true|2015-05-03| Sunday|
-122.40610663336099| 37.79361439203321| true|2015-05-03| Sunday|
-122.40610663336099| 37.79361439203321| true|2015-05-03| Sunday|
 -122.402043557609| 37.7971874633378| true|2015-05-03| Sunday|
-122.39858448694099|37.797986280621295| true|2015-05-03| Sunday|
 -122.404013943546| 37.7897518225049| true|2015-05-03| Sunday|
-122.40552479680001| 37.7977714997338| true|2015-05-03| Sunday|
-122.40552479680001| 37.7977714997338| true|2015-05-03| Sunday|
-122.40552479680001| 37.7977714997338| true|2015-05-03| Sunday|
-122.40583874581002| 37.7973631784416| true|2015-04-19| Sunday|
 -122.403535460378| 37.7878244847404| true|2015-04-19| Sunday|
-122.40031605728599| 37.790099627367| true|2015-04-19| Sunday|
 -122.402163713618| 37.7895710255863| true|2015-04-19| Sunday|
 -122.400922761049| 37.794959803128| true|2015-04-19| Sunday|
-122.40377224998899| 37.78912092500261| true|2015-04-19| Sunday|
 -122.400344691288| 37.7902143401629| true|2015-04-19| Sunday|
-122.40398013244801| 37.7964121687875| true|2015-04-19| Sunday|
 -122.396556426173| 37.7941169057941| true|2015-04-05| Sunday|
 -122.396556426173| 37.7941169057941| true|2015-04-05| Sunday|
+-------------------+------------------+-----------+----------+---------+
only showing top 20 rows

In [26]:
from pyspark.sql.functions import *
crime_month_df=df.groupBy(month(df.Dates).alias("Month"),year(df.Dates).alias("Year")).agg(count("*").alias("Numbers_Crime")).orderBy("Year","Month").select("Year","Month","Numbers_Crime")

crime_month_df.show()


+----+-----+-------------+
Year|Month|Numbers_Crime|
+----+-----+-------------+
2003| 1| 5492|
2003| 2| 5593|
2003| 3| 5684|
2003| 4| 6261|
2003| 5| 6567|
2003| 6| 5658|
2003| 7| 5135|
2003| 8| 5404|
2003| 9| 6269|
2003| 10| 6714|
2003| 11| 5753|
2003| 12| 4363|
2004| 1| 5554|
2004| 2| 5223|
2004| 3| 6783|
2004| 4| 6483|
2004| 5| 6317|
2004| 6| 5228|
2004| 7| 5276|
2004| 8| 5999|
+----+-----+-------------+
only showing top 20 rows

In [27]:

df_crime_month = spark.sql("SELECT YEAR(Dates) AS Year, MONTH(Dates) AS Month, COUNT(*) AS Numbers_Crime\
                            FROM sf_crime\
                            GROUP BY MONTH(Dates), YEAR(Dates)\
                            ORDER BY YEAR(Dates), MONTH(Dates)")
df_crime_month.show()


+----+-----+-------------+
Year|Month|Numbers_Crime|
+----+-----+-------------+
2003| 1| 5492|
2003| 2| 5593|
2003| 3| 5684|
2003| 4| 6261|
2003| 5| 6567|
2003| 6| 5658|
2003| 7| 5135|
2003| 8| 5404|
2003| 9| 6269|
2003| 10| 6714|
2003| 11| 5753|
2003| 12| 4363|
2004| 1| 5554|
2004| 2| 5223|
2004| 3| 6783|
2004| 4| 6483|
2004| 5| 6317|
2004| 6| 5228|
2004| 7| 5276|
2004| 8| 5999|
+----+-----+-------------+
only showing top 20 rows

In [28]:
df_crime_month = df_crime_month.toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [29]:

fig = plt.figure(figsize=(25,10))
sb.set(font_scale=2, style = "whitegrid", rc={"lines.linewidth": 2.5})
sb.barplot(x= "Month", y = "Numbers_Crime", hue="Year", data = df_crime_month)
plt.legend(loc='upper right')
display(fig)

In [30]:

crime_same_DF=df.where(to_date(df.Dates).isin('2003-01-06', '2004-01-10','2008-01-13','2015-05-13')).groupBy(hour(df.Dates).alias("Hour"),to_date(df.Dates).alias("Date"),year(df.Dates).alias("Year")).agg(count("*").alias("Numbers_Crime")).orderBy("Year","Date","Hour").select("Year","Date","Hour","Numbers_Crime")

crime_same_DF.show()

+----+----------+----+-------------+
Year| Date|Hour|Numbers_Crime|
+----+----------+----+-------------+
2003|2003-01-06| 0| 13|
2003|2003-01-06| 1| 6|
2003|2003-01-06| 2| 12|
2003|2003-01-06| 3| 3|
2003|2003-01-06| 4| 1|
2003|2003-01-06| 5| 8|
2003|2003-01-06| 6| 5|
2003|2003-01-06| 7| 10|
2003|2003-01-06| 8| 20|
2003|2003-01-06| 9| 15|
2003|2003-01-06| 10| 23|
2003|2003-01-06| 11| 25|
2003|2003-01-06| 12| 13|
2003|2003-01-06| 13| 16|
2003|2003-01-06| 14| 21|
2003|2003-01-06| 15| 12|
2003|2003-01-06| 16| 17|
2003|2003-01-06| 17| 27|
2003|2003-01-06| 18| 36|
2003|2003-01-06| 19| 22|
+----+----------+----+-------------+
only showing top 20 rows

In [31]:



df_crime_same = spark.sql("SELECT YEAR(Dates) AS Year, to_date(Dates) AS Date, HOUR(Dates) AS Hour,COUNT(*) AS Numbers_Crime\
                                FROM sf_crime\
                                WHERE to_date(Dates) in ('2003-01-06', '2004-01-10','2008-01-13','2015-05-13')\
                                 GROUP BY HOUR(Dates), to_date(Dates), YEAR(Dates)\
                                 ORDER BY YEAR(Dates), to_date(Dates), HOUR(Dates)")
                               


df_crime_same.show()

+----+----------+----+-------------+
Year| Date|Hour|Numbers_Crime|
+----+----------+----+-------------+
2003|2003-01-06| 0| 13|
2003|2003-01-06| 1| 6|
2003|2003-01-06| 2| 12|
2003|2003-01-06| 3| 3|
2003|2003-01-06| 4| 1|
2003|2003-01-06| 5| 8|
2003|2003-01-06| 6| 5|
2003|2003-01-06| 7| 10|
2003|2003-01-06| 8| 20|
2003|2003-01-06| 9| 15|
2003|2003-01-06| 10| 23|
2003|2003-01-06| 11| 25|
2003|2003-01-06| 12| 13|
2003|2003-01-06| 13| 16|
2003|2003-01-06| 14| 21|
2003|2003-01-06| 15| 12|
2003|2003-01-06| 16| 17|
2003|2003-01-06| 17| 27|
2003|2003-01-06| 18| 36|
2003|2003-01-06| 19| 22|
+----+----------+----+-------------+
only showing top 20 rows

In [32]:
df_crime_same_date=crime_same_DF.toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [33]:

fig = plt.figure(figsize=(30,15))
sb.set(font_scale=3, style = "whitegrid", rc={"lines.linewidth": 4.5})

for day in df_crime_same_date["Date"].unique():
  plt.stackplot(df_crime_same_date[df_crime_same_date["Date"] == day]["Hour"],
                df_crime_same_date[df_crime_same_date["Date"] == day]["Numbers_Crime"] , alpha=0.3)

plt.legend([2003, 2004, 2008, 2015], loc='upper left')
display(fig)

In [34]:
crime_PdDis_df=df.groupBy("PdDistrict").agg(count("*").alias("Numbers_Crime")).orderBy("Numbers_Crime",ascending=False)
crime_PdDis_df.show()

+----------+-------------+
PdDistrict|Numbers_Crime|
+----------+-------------+
 SOUTHERN| 155542|
 MISSION| 119898|
 NORTHERN| 105169|
 BAYVIEW| 89355|
 CENTRAL| 85455|
TENDERLOIN| 81794|
 INGLESIDE| 78841|
 PARK| 49269|
 RICHMOND| 26621|
 TARAVAL| 25833|
+----------+-------------+

In [35]:
df.createOrReplaceTempView("sf_crime")

In [36]:

df_crime_danger_dist = spark.sql("SELECT PdDistrict, COUNT(*) AS Numbers_Crime \
                                  FROM sf_crime \
                                  GROUP BY PdDistrict \
                                  ORDER BY Numbers_Crime DESC")

df_crime_danger_dist.show()

+----------+-------------+
PdDistrict|Numbers_Crime|
+----------+-------------+
 SOUTHERN| 155542|
 MISSION| 119898|
 NORTHERN| 105169|
 BAYVIEW| 89355|
 CENTRAL| 85455|
TENDERLOIN| 81794|
 INGLESIDE| 78841|
 PARK| 49269|
 RICHMOND| 26621|
 TARAVAL| 25833|
+----------+-------------+

In [37]:
from pyspark.sql.functions import desc,max,col,sum,rank,asc

crime_danger_cat_DF=df.where(df.PdDistrict.isin('SOUTHERN','MISSION','NORTHERN')).groupBy(df.Category,hour(df.Dates).alias("Hour")).agg(count("*").alias("Numbers_Crime")).orderBy(asc("Hour"),desc("Numbers_Crime")).select("Category","Hour","Numbers_Crime")

crime_danger_cat_DF.show()

+--------------------+----+-------------+
 Category|Hour|Numbers_Crime|
+--------------------+----+-------------+
 LARCENY/THEFT| 0| 3569|
 OTHER OFFENSES| 0| 3000|
 NON-CRIMINAL| 0| 2091|
 ASSAULT| 0| 1922|
 VANDALISM| 0| 1101|
 FRAUD| 0| 1026|
 SUSPICIOUS OCC| 0| 853|
 WARRANTS| 0| 771|
 DRUG/NARCOTIC| 0| 767|
FORGERY/COUNTERFE...| 0| 734|
 VEHICLE THEFT| 0| 656|
 BURGLARY| 0| 618|
 ROBBERY| 0| 556|
 PROSTITUTION| 0| 525|
 MISSING PERSON| 0| 349|
SEX OFFENSES FORC...| 0| 245|
 SECONDARY CODES| 0| 208|
 DRUNKENNESS| 0| 184|
 WEAPON LAWS| 0| 172|
 DISORDERLY CONDUCT| 0| 129|
+--------------------+----+-------------+
only showing top 20 rows

In [38]:
df_crime_danger_cat = spark.sql("SELECT \
                                    Category, \
                                    HOUR(Dates) AS Hour, \
                                    COUNT(*) AS Numbers_Crime \
                                 FROM sf_crime \
                                 WHERE PdDistrict IN ('SOUTHERN','MISSION','NORTHERN') \
                                 GROUP BY Category, HOUR(Dates) \
                                 ORDER BY HOUR(Dates), Numbers_Crime DESC")

df_crime_danger_cat.show()


+--------------------+----+-------------+
 Category|Hour|Numbers_Crime|
+--------------------+----+-------------+
 LARCENY/THEFT| 0| 3569|
 OTHER OFFENSES| 0| 3000|
 NON-CRIMINAL| 0| 2091|
 ASSAULT| 0| 1922|
 VANDALISM| 0| 1101|
 FRAUD| 0| 1026|
 SUSPICIOUS OCC| 0| 853|
 WARRANTS| 0| 771|
 DRUG/NARCOTIC| 0| 767|
FORGERY/COUNTERFE...| 0| 734|
 VEHICLE THEFT| 0| 656|
 BURGLARY| 0| 618|
 ROBBERY| 0| 556|
 PROSTITUTION| 0| 525|
 MISSING PERSON| 0| 349|
SEX OFFENSES FORC...| 0| 245|
 SECONDARY CODES| 0| 208|
 DRUNKENNESS| 0| 184|
 WEAPON LAWS| 0| 172|
 DISORDERLY CONDUCT| 0| 129|
+--------------------+----+-------------+
only showing top 20 rows

In [39]:
df_crime_danger_cat = df_crime_danger_cat.toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [40]:

fig = plt.figure(figsize=(50,20))
sb.set(font_scale=3, style = "whitegrid")
sb.barplot(x= "Hour", y = "Numbers_Crime",  data = df_crime_danger_cat)
plt.legend(loc='upper left')
display(fig)

In [41]:
fig = plt.figure(figsize=(30,40))
sb.set(font_scale=2, style = "whitegrid")
sb.barplot(x= "Numbers_Crime", y = "Category", data = df_crime_danger_cat)
plt.legend(loc='upper left')
plt.xticks(rotation = 0)
display(fig)

In [42]:
from pyspark.sql.functions import count
crime_percent_df=df.groupBy("Category").agg(count("*").alias("Numbers_Crime")).orderBy("Numbers_Crime",ascending=False)
crime_percent_df.show()

+--------------------+-------------+
 Category|Numbers_Crime|
+--------------------+-------------+
 LARCENY/THEFT| 163120|
 OTHER OFFENSES| 118452|
 NON-CRIMINAL| 85169|
 ASSAULT| 72098|
 DRUG/NARCOTIC| 52851|
 VEHICLE THEFT| 47861|
 WARRANTS| 40997|
 VANDALISM| 40127|
 BURGLARY| 33429|
 SUSPICIOUS OCC| 28383|
 MISSING PERSON| 24104|
 ROBBERY| 21914|
 FRAUD| 15138|
FORGERY/COUNTERFE...| 9595|
 SECONDARY CODES| 9088|
 WEAPON LAWS| 8094|
 PROSTITUTION| 7393|
 TRESPASS| 6971|
 STOLEN PROPERTY| 4328|
 DISORDERLY CONDUCT| 4166|
+--------------------+-------------+
only showing top 20 rows

In [43]:
df_crime_percent= spark.sql("SELECT Category, COUNT(*) AS Numbers_Crime \
                                  FROM sf_crime \
                                  GROUP BY Category")


df_crime_percent.show(5)

+--------------------+-------------+
 Category|Numbers_Crime|
+--------------------+-------------+
 FRAUD| 15138|
 SUICIDE| 419|
SEX OFFENSES FORC...| 4045|
 LIQUOR LAWS| 1784|
 SECONDARY CODES| 9088|
+--------------------+-------------+
only showing top 5 rows

In [44]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window
df_crime_percent_cat  = crime_percent_df.withColumn('Crime_Percent', f.col('Numbers_Crime')*100/f.sum('Numbers_Crime').over(Window.partitionBy()))
df_crime_percent_cat=df_crime_percent_cat.orderBy('Crime_Percent', ascending=False).show()

+--------------------+-------------+------------------+
 Category|Numbers_Crime| Crime_Percent|
+--------------------+-------------+------------------+
 LARCENY/THEFT| 163120|19.946758101536236|
 OTHER OFFENSES| 118452|14.484633341363232|
 NON-CRIMINAL| 85169|10.414697405282858|
 ASSAULT| 72098| 8.81633990684502|
 DRUG/NARCOTIC| 52851| 6.462764298824741|
 VEHICLE THEFT| 47861| 5.852573501088927|
 WARRANTS| 40997| 5.013224876708443|
 VANDALISM| 40127| 4.906838905960916|
 BURGLARY| 33429| 4.087789213929959|
 SUSPICIOUS OCC| 28383| 3.470750583594305|
 MISSING PERSON| 24104|2.9475028033314707|
 ROBBERY| 21914| 2.679703635587697|
 FRAUD| 15138|1.8511158910069616|
FORGERY/COUNTERFE...| 9595|1.1733027463477208|
 SECONDARY CODES| 9088|1.1113054047741622|
 WEAPON LAWS| 8094|0.9897563761269882|
 PROSTITUTION| 7393|0.9040361859039812|
 TRESPASS| 6971|0.8524328759551809|
 STOLEN PROPERTY| 4328|0.5292396337876952|
 DISORDERLY CONDUCT| 4166|0.5094298323381558|
+--------------------+-------------+------------------+
only showing top 20 rows